In [22]:
print("Sc Vax Classify")

Sc Vax Classify


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_classif
from sklearn.metrics import accuracy_score

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.python.keras import models
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout

In [24]:
#get the data
df = pd.read_csv('train_val.csv')
df.head()

,ID,tweet,labels
0,1296010336907038720t,@cath__kath AstraZeneca is made with the kidne...,ingredients
1,1336808189677940736t,It begins. Please find safe alternatives to th...,side-effect
2,1329488407307956231t,"@PaolaQP1231 Well, I mean congratulations Covi...",side-effect
3,1364194604459900934t,@BorisJohnson for those of us that do not wish...,mandatory
4,1375938799247765515t,She has been trying to speak out: writing lett...,side-effect rushed


In [25]:
#std defn
num_classes = 2

In [26]:
#split data for each class
'''[unnecessary]: The tweet indicates vaccines are unnecessary, or that alternate cures are better.
 - [mandatory]: Against mandatory vaccination — The tweet suggests that vaccines should not be made mandatory.
 - [pharma]: Against Big Pharma — The tweet indicates that the Big Pharmaceutical companies are just trying to earn money, or the tweet is against such companies in general because of their history.
 - [conspiracy]: Deeper Conspiracy — The tweet suggests some deeper conspiracy, and not just that the Big Pharma want to make money (e.g., vaccines are being used to track people, COVID is a hoax)
 - [political]: Political side of vaccines — The tweet expresses concerns that the governments / politicians are pushing their own agenda though the vaccines.
 - [country]: Country of origin — The tweet is against some vaccine because of the country where it was developed / manufactured
 - [rushed]: Untested / Rushed Process — The tweet expresses concerns that the vaccines have not been tested properly or that the published data is not accurate.
 - [ingredients]: Vaccine Ingredients / technology — The tweet expresses concerns about the ingredients present in the vaccines (eg. fetal cells, chemicals) or the technology used (e.g., mRNA vaccines can change your DNA)
 - [side-effect]: Side Effects / Deaths — The tweet expresses concerns about the side effects of the vaccines, including deaths caused.
 - [ineffective]: Vaccine is ineffective — The tweet expresses concerns that the vaccines are not effective enough and are useless.
 - [religious]: Religious Reasons — The tweet is against vaccines because of religious reasons
 - [none]: No specific reason stated in the tweet, or some reason other than the given ones.']'''

attr = ['unnecessary', 'mandatory', 'pharma', 'conspiracy', 'political', 'country', 'rushed', 'ingredients', 'side-effect', 'ineffective', 'religious', 'none']

#split data for each class

# copy the df and modify such that for those rows where the string value of labels column contains 'unnecessary', set the label columnn to 1 else 0
# make a copy of the df and set it to another variable
df_unnecessary = df.copy()
df_unnecessary['unnecessary'] = df_unnecessary['labels'].apply(lambda x: 1 if 'unnecessary' in x else 0)

# copy the df and modify such that for those rows where the string value of labels column contains 'mandatory', set the label columnn to 1 else 0
df_mandatory = df.copy()
df_mandatory['mandatory'] = df_mandatory['labels'].apply(lambda x: 1 if 'mandatory' in x else 0)

# copy the df and modify such that for those rows where the string value of labels column contains 'pharma', set the label columnn to 1 else 0
df_pharma = df.copy()
df_pharma['pharma'] = df_pharma['labels'].apply(lambda x: 1 if 'pharma' in x else 0)

# copy the df and modify such that for those rows where the string value of labels column contains 'conspiracy', set the label columnn to 1 else 0
df_conspiracy = df.copy()
df_conspiracy['conspiracy'] = df_conspiracy['labels'].apply(lambda x: 1 if 'conspiracy' in x else 0)

# copy the df and modify such that for those rows where the string value of labels column contains 'political', set the label columnn to 1 else 0
df_political = df.copy()
df_political['political'] = df_political['labels'].apply(lambda x: 1 if 'political' in x else 0)

# copy the df and modify such that for those rows where the string value of labels column contains 'country', set the label columnn to 1 else 0
df_country = df.copy()
df_country['country'] = df_country['labels'].apply(lambda x: 1 if 'country' in x else 0)

# copy the df and modify such that for those rows where the string value of labels column contains 'rushed', set the label columnn to 1 else 0
df_rushed = df.copy()
df_rushed['rushed'] = df_rushed['labels'].apply(lambda x: 1 if 'rushed' in x else 0)

# copy the df and modify such that for those rows where the string value of labels column contains 'ingredients', set the label columnn to 1 else 0
df_ingredients = df.copy()
df_ingredients['ingredients'] = df_ingredients['labels'].apply(lambda x: 1 if 'ingredients' in x else 0)

# copy the df and modify such that for those rows where the string value of labels column contains 'side-effect', set the label columnn to 1 else 0
df_side_effect = df.copy()
df_side_effect['side-effect'] = df_side_effect['labels'].apply(lambda x: 1 if 'side-effect' in x else 0)

# copy the df and modify such that for those rows where the string value of labels column contains 'ineffective', set the label columnn to 1 else 0
df_ineffective = df.copy()
df_ineffective['ineffective'] = df_ineffective['labels'].apply(lambda x: 1 if 'ineffective' in x else 0)

# copy the df and modify such that for those rows where the string value of labels column contains 'religious', set the label columnn to 1 else 0
df_religious = df.copy()
df_religious['religious'] = df_religious['labels'].apply(lambda x: 1 if 'religious' in x else 0)

# copy the df and modify such that for those rows where the string value of labels column contains 'none', set the label columnn to 1 else 0
df_none = df.copy()
df_none['none'] = df_none['labels'].apply(lambda x: 1 if 'none' in x else 0)

df_all = [df_unnecessary, df_mandatory, df_pharma, df_conspiracy, df_political, df_country, df_rushed, df_ingredients, df_side_effect, df_ineffective, df_religious, df_none]

In [27]:
def ngram_vectorize(train_texts, train_labels, val_texts):
        # Vectorization parameters
    # Range (inclusive) of n-gram sizes for tokenizing text.
        NGRAM_RANGE = (1, 2)

        # Limit on the number of features. We use the top 20K features.
        TOP_K = 20000

        # Whether text should be split into word or character n-grams.
        # One of 'word', 'char'.
        TOKEN_MODE = 'word'

        # Minimum document/corpus frequency below which a token will be discarded.
        MIN_DOCUMENT_FREQUENCY = 2
        """Vectorizes texts as n-gram vectors.

        1 text = 1 tf-idf vector the length of vocabulary of unigrams + bigrams.

        # Arguments
            train_texts: list, training text strings.
            train_labels: np.ndarray, training labels.
            val_texts: list, validation text strings.

        # Returns
            x_train, x_val: vectorized training and validation texts
        """
        # Create keyword arguments to pass to the 'tf-idf' vectorizer.
        kwargs = {
                'ngram_range': NGRAM_RANGE,  # Use 1-grams + 2-grams.
                'dtype': np.float64,
                'strip_accents': 'unicode',
                'decode_error': 'replace',
                'analyzer': TOKEN_MODE,  # Split text into word tokens.
                'min_df': MIN_DOCUMENT_FREQUENCY,
        }
        vectorizer = TfidfVectorizer(**kwargs)

        # Learn vocabulary from training texts and vectorize training texts.
        x_train = vectorizer.fit_transform(train_texts)

        # Vectorize validation texts.
        x_val = vectorizer.transform(val_texts)

        # Select top 'k' of the vectorized features.
        selector = SelectKBest(f_classif, k=min(TOP_K, x_train.shape[1]))
        selector.fit(x_train, train_labels)
        x_train = selector.transform(x_train).astype('float32')
        x_val = selector.transform(x_val).astype('float32')
        return x_train, x_val

In [28]:
def _get_last_layer_units_and_activation(num_classes):
    """Gets the # units and activation function for the last network layer.

    # Arguments
        num_classes: int, number of classes.

    # Returns
        units, activation values.
    """
    if num_classes == 2:
        activation = 'sigmoid'
        units = 1
    else:
        activation = 'softmax'
        units = num_classes
    return units, activation

In [29]:
def mlp_model(layers, units, dropout_rate, input_shape, num_classes):
    """Creates an instance of a multi-layer perceptron model.

    # Arguments
        layers: int, number of `Dense` layers in the model.
        units: int, output dimension of the layers.
        dropout_rate: float, percentage of input to drop at Dropout layers.
        input_shape: tuple, shape of input to the model.
        num_classes: int, number of output classes.

    # Returns
        An MLP model instance.
    """
    op_units, op_activation = _get_last_layer_units_and_activation(num_classes)
    model = models.Sequential()
    model.add(Dropout(rate=dropout_rate, input_shape=input_shape))

    for _ in range(layers-1):
        model.add(Dense(units=units, activation='relu'))
        model.add(Dropout(rate=dropout_rate))

    model.add(Dense(units=op_units, activation=op_activation))
    return model

we have a bunch of options in front of us: binary chain, ensembling (best but maybe complex to choose b/w ensembling methods), power set (not rec)

In [30]:
def train_ngram_model(
    data,
    learning_rate=10e-3,
    epochs=1000,
    batch_size=128,
    layers=2,
    units=64,
    dropout_rate=0.2,
):
    """Trains n-gram model on the given dataset.

    # Arguments
        data: tuples of training and test texts and labels.
        learning_rate: float, learning rate for training model.
        epochs: int, number of epochs.
        batch_size: int, number of samples per batch.
        layers: int, number of `Dense` layers in the model.
        units: int, output dimension of Dense layers in the model.
        dropout_rate: float: percentage of input to drop at Dropout layers.

    # Raises
        ValueError: If validation data has label values which were not seen
            in the training data.
    """
    # Get the data.
    (train_texts, train_labels), (val_texts, val_labels) = data

    # Verify that validation labels are in the same range as training labels.
    # num_classes = get_num_classes(train_labels)
    # unexpected_labels = [v for v in val_labels if v not in range(num_classes)]
    # if len(unexpected_labels):
    #     raise ValueError('Unexpected label values found in the validation set:'
    #                      ' {unexpected_labels}. Please make sure that the '
    #                      'labels in the validation set are in the same range '
    #                      'as training labels.'.format(
    #                          unexpected_labels=unexpected_labels))

    # Vectorize texts.
    x_train, x_val = ngram_vectorize(train_texts, train_labels, val_texts)

    # Create model instance.
    model = mlp_model(
        layers=layers,
        units=units,
        dropout_rate=dropout_rate,
        input_shape=x_train.shape[1:],
        num_classes=2,
    )

    # Compile model with learning parameters.
    if num_classes == 2:
        loss = "binary_crossentropy"
    else:
        loss = "sparse_categorical_crossentropy"

    optimizer = tf.keras.optimizers.legacy.Adam(lr=learning_rate)
    
    model.compile(optimizer="Adam", loss=loss, metrics=["acc"])

    # Create callback for early stopping on validation loss. If the loss does
    # not decrease in two consecutive tries, stop training.
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)]

    predictions = model.predict(val_texts)
    accuracy = accuracy_score(val_labels, predictions)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))
    model.save("tweet_vax.h5")
    # return history["val_acc"][-1], history["val_loss"][-1]
    

In [31]:
for i in df_all:
    # Split the data into training and testing sets
    train_texts, val_texts, train_labels, val_labels = train_test_split(i.iloc[:, 1], i.iloc[:, -1], test_size=0.2, random_state=42)
    data  = (train_texts, train_labels), (val_texts, val_labels)
    
    train_ngram_model(data)    

/Users/kesh/Library/Python/3.9/lib/python/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


AttributeError: module 'tensorflow.python.distribute.input_lib' has no attribute 'DistributedDatasetInterface'